In [1]:
#lstm-10days
#인풋데이터가 잘못되어 revision 


import pandas as pd
import numpy as np
'''
data preprocessing
'''

#주가데이터 파일을 로드
csv_data = pd.read_csv("./data/005930_200325.csv")
csv_data

,date,start,high,low,end,volume,foreign,person,company,wondollar
0,2020.3.25,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9
1,2020.3.24,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6
2,2020.3.23,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5
3,2020.3.20,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5
4,2020.3.19,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7
...,...,...,...,...,...,...,...,...,...,...
592,2017.10.23,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2
593,2017.10.20,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0
594,2017.10.19,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4
595,2017.10.18,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9


In [2]:
#필요한 부분을 가져온다
stdata = csv_data[["start","high", "low","end","volume","foreign", "person","company","wondollar"]]
stdata.head()

#전일대비 가격 변화율을 추가한다
price_change = -np.diff(stdata["end"].to_numpy())
price = stdata["end"].to_numpy()
price = price[1:]
price_change_rate = (price_change/price)*100
price_change_rate=np.append(price_change_rate,[0])
stdata['price_change_rate'] = price_change_rate

#캔들 (종가,시가,고가,저가) 관련 정보를 추가한다
#시가-종가
start = stdata["start"].to_numpy()
end = stdata["end"].to_numpy()
candle_end_start = end-start
stdata['candle_end_start'] = candle_end_start

#고가-저가 (당일변동폭) 
high = stdata["high"].to_numpy()
low = stdata["low"].to_numpy()
candle_high_low = high-low
stdata['candle_high_low'] = candle_high_low


#상승or하락 표시 상승:1 보함:0 하락:-1
up_or_down = np.zeros(597)

price_change_rate = stdata["price_change_rate"].to_numpy()
for idx, val in enumerate(price_change_rate):
    if val>0:
        up_or_down[idx]=1
    if val<0:
        up_or_down[idx]=-1
stdata['up_or_down'] = up_or_down

stdata


,start,high,low,end,volume,foreign,person,company,wondollar,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9,3.620873,-300,2450,1.0
1,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6,10.470588,3100,3900,1.0
2,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5,-6.387665,-100,1150,-1.0
3,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5,5.704307,1250,1950,1.0
4,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7,-5.811404,-3450,4350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2,0.854383,-300,640,1.0
593,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0,1.623254,1040,1300,1.0
594,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4,-3.250548,-1720,1720,-1.0
595,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9,-0.072993,-60,1200,-1.0


In [3]:
#사용할 feature를 선택한다

input_data = stdata[["price_change_rate","candle_end_start","candle_high_low","up_or_down"]]
input_data

,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,3.620873,-300,2450,1.0
1,10.470588,3100,3900,1.0
2,-6.387665,-100,1150,-1.0
3,5.704307,1250,1950,1.0
4,-5.811404,-3450,4350,-1.0
...,...,...,...,...
592,0.854383,-300,640,1.0
593,1.623254,1040,1300,1.0
594,-3.250548,-1720,1720,-1.0
595,-0.072993,-60,1200,-1.0


In [4]:
#정규화 안할경우
input_arr = input_data.to_numpy()
input_arr.shape

(597, 4)

In [5]:
#정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fitted = scaler.fit(input_data)
input_arr = scaler.transform(input_data)
input_arr.shape

(597, 4)

In [6]:
#역순
input_arr = np.flip(input_arr, axis=0)
print(input_arr.shape)
#input_arr

(597, 4)


In [7]:
#정답추출
up_or_down_ans = input_arr[:,3]

up_or_down_ans = up_or_down_ans[1:]
print(up_or_down_ans.shape)
#up_or_down_ans

(596,)


In [8]:
#마지막 행은 정답이 없어 버린다
input_arr = input_arr[:-1]
print(input_arr.shape)
#input_arr

(596, 4)


In [9]:
#1일 -> len_of_day일 단위의 데이터로 변환 
len_of_day =10
feature_count = input_arr.shape[1]




#596일의 데이터로 10일짜리 데이터를 생성하면 596-10+1=587 개의 데이터가 생성된다
input_data_count = input_arr.shape[0]-len_of_day+1



#596*6열     [[feature갯수 개], 
#             [feature갯수개], 
#             [feature갯수개],
#             [feature갯수개],
#              ... ]
#
# 
#              을 [    [ [feature갯수],[feature갯수],[feature갯수]..........[feature갯수] (feature갯수개짜리 n일) ],
#                [ (feature갯수짜리 n일개) ],
#                [ (13개짜리 n일개) ],
#               .
#               .
#               . 
#               582개 
#    
#            ]  ( 582  ,10(일수) , 6(feature갯수) )    형태로 만들어야 한다


#inputdata1 : 10일 짜리 데이터 생성 

inputdata1 = np.empty((input_data_count,len_of_day,feature_count), dtype=np.float64)

for i in range(input_data_count):
    for j in range(len_of_day):
        inputdata1[i][j]=input_arr[i+j]
        
inputdata1.shape 

(587, 10, 4)

In [10]:
up_or_down_ans = up_or_down_ans[len_of_day - 1:]
print(up_or_down_ans.shape)
up_or_down_ans=up_or_down_ans.reshape(-1,1)
print(up_or_down_ans.shape)
#up_or_down_ans

(587,)
(587, 1)


In [11]:
#up_or_down_ans

In [12]:
for idx, data in enumerate(up_or_down_ans) :
    if data <0.6:
        up_or_down_ans[idx] = 0
#up_or_down_ans       

In [13]:
up_or_down_ans = up_or_down_ans.astype(np.int)

In [14]:
#입력 : inputdata1   (587, 10, 6))
#정답 : up_or_down_ans (587,)

In [15]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [16]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [17]:
#train : 500 test : 87
inputdata1_train = inputdata1[:500]
inputdata1_test = inputdata1[500:]

up_or_down_train = up_or_down_ans[:500]
up_or_down_test =up_or_down_ans[500:]

In [18]:
class TrainDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_train
    self.y_data = up_or_down_train

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [19]:
class TestDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_test
    self.y_data = up_or_down_test

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [20]:
train_dataset = TrainDataset()
test_dataset = TestDataset()
dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [21]:
# feature 갯수
input_size = feature_count

#output size 바로 결과 예측을 위해서 1로 설정 
hidden_dim = 2

batch_size = 1 

# 15일
sequence_length = len_of_day

# one layer rnn이다.
n_layers = 1

#inputs = (batch_size * seq_len * input_size ) 인 행렬\
#hidden = (num_layers * batch_size * hidden_size ) 인 행렬
#output = (batch,seq_len,hidden)

In [22]:
#상승 또는 하락 판별 모델
class Model(nn.Module):
    def __init__(self, input_size, hidden_dim,output_size ,n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        #self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        h0 = self.init_hidden(batch_size)
        #c0 = self.init_hidden(batch_size)
        

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hn = self.rnn(x, h0)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        
        #out = self.fc(out)
        #out = out[:,-1,:].view(1,1,1)
        return out[:,-1,:],
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.rand(self.n_layers, batch_size, self.hidden_dim)
        hidden = nn.init.xavier_normal_(hidden).to(device)
        return hidden

In [63]:
#모델 초기화
model = Model(input_size, hidden_dim, output_size=1, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
nb_epochs = 200
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [64]:
#학습
#for epoch in range(nb_epochs + 1):
#     for batch_idx, samples in enumerate(dataloader):
for epoch in range(nb_epochs + 1):
    print("epoch: ", epoch)
    lossval=0
    for batch_idx, (input_data,label) in enumerate(dataloader):
        optimizer.zero_grad()
        input_data = input_data.float().to(device)
        label = label.view(-1).long().to(device)
        output, = model(input_data)
        #print(input_data.shape)
        #print(output.shape)
        #print(label.shape)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if(batch_idx%10==0):
            #print("epoch: ",epoch, "  iter:",batch_idx,"/502")
            #print("Loss: {:.4f}".format(loss.item()))
            lossval += loss.item()
            #print("")
    print("lossval: ", lossval/55.0 )

epoch:  0
lossval:  0.6359643361785196
epoch:  1
lossval:  0.6359624288298867
epoch:  2
lossval:  0.6324278625574979
epoch:  3
lossval:  0.6262637745250356
epoch:  4
lossval:  0.6343832937153903
epoch:  5
lossval:  0.6298519058661027
epoch:  6
lossval:  0.6321472644805908
epoch:  7
lossval:  0.6301768996498801
epoch:  8
lossval:  0.6327909751371904
epoch:  9
lossval:  0.6325356429273432
epoch:  10
lossval:  0.6308992776003751
epoch:  11
lossval:  0.6286203752864491
epoch:  12
lossval:  0.6306581648913296
epoch:  13
lossval:  0.6313389095393094
epoch:  14
lossval:  0.628349460255016
epoch:  15
lossval:  0.6300098765980113
epoch:  16
lossval:  0.6260462251576511
epoch:  17
lossval:  0.6307453036308288
epoch:  18
lossval:  0.632328147237951
epoch:  19
lossval:  0.6327377275987105
epoch:  20
lossval:  0.6324427983977577
epoch:  21
lossval:  0.6123044333674691
epoch:  22
lossval:  0.6258321458643133
epoch:  23
lossval:  0.6353093851696361
epoch:  24
lossval:  0.6184167818589644
epoch:  25
l

In [65]:
#train acc
trainsize= 500.0
with torch.no_grad():
    count = 0
    for i in range(int(trainsize)):
        ip, ans = train_dataset.__getitem__(i)
        ip = ip.float().view(-1,10,feature_count).to(device)
        output, = model(ip)
        a,p = torch.max(output,1)
        if(p.item() == ans.item()):
            count += 1
            #print("right")
    acc = count/trainsize
    print(acc)

0.522


In [66]:
#test acc 
testsize= 80.0
with torch.no_grad():
    count = 0
    for i in range(int(testsize)):
        ip, ans = train_dataset.__getitem__(i)
        ip = ip.float().view(-1,10,feature_count).to(device)
        output, = model(ip)
        a,p = torch.max(output,1)
        if(p.item() == ans.item()):
            count += 1
            #print("right")
    acc = count/testsize
    print(acc)

0.575
